# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [2]:
income = pd.read_csv("census_data.csv")

In [3]:
income.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [7]:
income.dtypes.sort_values()

age                int64
education_num      int64
capital_gain       int64
capital_loss       int64
hours_per_week     int64
workclass         object
education         object
marital_status    object
occupation        object
relationship      object
race              object
gender            object
native_country    object
income_bracket    object
dtype: object

In [5]:
cat_col = income.columns[income.dtypes!=np.int64]
cat_col

Index(['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'gender', 'native_country', 'income_bracket'],
      dtype='object')

In [10]:
num_col = income.columns[income.dtypes ==np.int64]
num_col

Index(['age', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')

In [11]:
income.income_bracket.unique()

array([' <=50K', ' >50K'], dtype=object)

In [12]:
income.income_bracket = income.income_bracket.map({' <=50K':0 ,' >50K':1 })

### Perform a Train Test Split on the Data

In [13]:
X = income.drop("income_bracket",1)

In [14]:
y = income.income_bracket

In [15]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X, y , test_size = 0.3 , random_state = 101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [16]:
cat_col = cat_col.drop("income_bracket")

In [17]:
cat_col

Index(['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'gender', 'native_country'],
      dtype='object')

In [18]:
new_cat_col = []
for i in cat_col:
    new_cat_col.append(i+"_")

In [69]:
feat_cols = []
test_cat_col= {}
for name , col in zip( new_cat_col , cat_col):
    test_cat_col[name] = tf.feature_column.categorical_column_with_hash_bucket(col,hash_bucket_size=50)
    test_cat_col[name] = tf.feature_column.embedding_column(test_cat_col[name],dimension = 50)
    feat_cols.append(test_cat_col[name])

In [70]:
feat_cols

[_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='workclass', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C550>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='education', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C0F0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='marital_status', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C470>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical

In [20]:
new_num_col = []
for i in num_col:
    new_num_col.append(i+"_")

In [71]:
test_num_col = {}
for name , col in zip(new_num_col, num_col):
    test_num_col[name] = tf.feature_column.numeric_column(col)
    feat_cols.append(test_num_col[name])

In [72]:
feat_cols

[_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='workclass', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C550>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='education', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C0F0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='marital_status', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C470>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical

** Import Tensorflow **

In [22]:
globals().update(test_num_col)
globals().update(test_cat_col)

In [34]:
for i in test_cat_col.keys():
    print(i)

workclass_
education_
marital_status_
occupation_
relationship_
race_
gender_
native_country_


In [38]:
workclass_

_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='workclass', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F3093A20>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

In [37]:
workclass_ = tf.feature_column.embedding_column(workclass_, dimension=50)
education_ = tf.feature_column.embedding_column(education_,dimension=50)
marital_status_ = tf.feature_column.embedding_column(marital_status_,dimension=50)
occupation_ = tf.feature_column.embedding_column(occupation_,dimension=50)
relationship_ = tf.feature_column.embedding_column(relationship_,dimension=50)
race_ = tf.feature_column.embedding_column(race_,dimension=50)
gender_ = tf.feature_column.embedding_column(gender_,dimension=50)
native_country_ = tf.feature_column.embedding_column(native_country_,dimension=50)

In [23]:
test_cat_col.keys()

dict_keys(['workclass_', 'education_', 'marital_status_', 'occupation_', 'relationship_', 'race_', 'gender_', 'native_country_'])

In [39]:
feat_cols = [age_, capital_loss_,capital_gain_,education_num_,hours_per_week_,
            workclass_, education_, marital_status_,occupation_,relationship_,race_,gender_,native_country_]

In [73]:
for i in feat_cols:
    print(i)

_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='workclass', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C550>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)
_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='education', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C0F0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)
_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='marital_status', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F524C470>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)
_EmbeddingColumn(categorical_column

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

** Create the continuous feature_columns for the continuous values using numeric_column **

** Put all these variables into a single list with the variable name feat_cols **

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [74]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train , y_train , batch_size = 10, num_epochs=1000,shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [42]:
units = []
for i in range(3):
    units.append(13)

In [43]:
len(units)

3

In [75]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=units,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Mostafa\\AppData\\Local\\Temp\\tmp89t2ox1t', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [45]:
feat_cols

[_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='capital_loss', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='capital_gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='education_num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='hours_per_week', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='workclass', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x000001A4F3093A20>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='education', hash_bucket_size=50, dtype=tf.string), dimension=50, combiner='mean', 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_checkpoints_secs': 600, '_model_dir': 'C:\\Users\\Marcial\\AppData\\Local\\Temp\\tmpyyyqnxgv', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_tf_random_seed': 1}


** Train your model on the data, for at least 5000 steps. **

In [76]:
dnn_model.train(input_fn=input_func,steps = 5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Mostafa\AppData\Local\Temp\tmp89t2ox1t\model.ckpt.
INFO:tensorflow:loss = 7.69124, step = 1
INFO:tensorflow:global_step/sec: 274.657
INFO:tensorflow:loss = 6.19514, step = 101 (0.366 sec)
INFO:tensorflow:global_step/sec: 331.343
INFO:tensorflow:loss = 4.21423, step = 201 (0.303 sec)
INFO:tensorflow:global_step/sec: 335.805
INFO:tensorflow:loss = 5.14295, step = 301 (0.297 sec)
INFO:tensorflow:global_step/sec: 343.912
INFO:tensorflow:loss = 3.15695, step = 401 (0.290 sec)
INFO:tensorflow:global_step/sec: 330.245
INFO:tensorflow:loss = 3.9846, step = 501 (0.303 sec)
INFO:tensorflow:global_step/sec: 356.192
INFO:tensorflow:loss = 2.32466, step = 601 (0.282 sec)
INFO:tensorflow:global_step/sec: 297.714
INFO:tensorflow:loss = 3.8014, step = 701 (0.335 sec)
INFO:tensorflow:global_step/sec: 348.722
INFO:tensorflow:loss = 3.12984, step = 801 (0.288 sec)
INFO:tensorflow:global_step/sec: 335.804
IN

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [77]:
eval_func = tf.estimator.inputs.pandas_input_fn(X_test  , batch_size=10,shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [78]:
 result = dnn_model.predict(eval_func)

In [79]:
result = list(result)

INFO:tensorflow:Restoring parameters from C:\Users\Mostafa\AppData\Local\Temp\tmp89t2ox1t\model.ckpt-5000


In [80]:
result[0]


{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.24901748], dtype=float32),
 'logits': array([-1.10385931], dtype=float32),
 'probabilities': array([ 0.75098246,  0.24901746], dtype=float32)}

** Each item in your list will look like this: **

{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.21327116], dtype=float32),
 'logits': array([-1.30531931], dtype=float32),
 'probabilities': array([ 0.78672886,  0.21327116], dtype=float32)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [81]:
class_id = []
for i in result:
    class_id.append(int(i["class_ids"]))

In [82]:
class_id[0:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [83]:
from sklearn.metrics import classification_report

In [84]:
report = classification_report(y_test, class_id)

In [85]:
print(report)

             precision    recall  f1-score   support

          0       0.85      0.95      0.90      7436
          1       0.74      0.47      0.57      2333

avg / total       0.82      0.83      0.82      9769



             precision    recall  f1-score   support

          0       0.88      0.92      0.90      7436
          1       0.70      0.59      0.64      2333

avg / total       0.84      0.84      0.84      9769



# Great Job!